In [ ]:
!nvidia-smi

Fri Apr  4 14:24:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   60C    P8             15W /   72W |       1MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%%capture
!pip install datasets -q
!pip install sacrebleu -q
!pip install tqdm -q
!pip install numpy -q
!pip install huggingface_hub -q
!pip install faster-whisper -q
!pip install transformers -q
!pip install evaluate -q
!pip install jiwer -q
!pip install peft -q
!pip install accelerate
!pip install flash-attn --no-build-isolation

In [ ]:
from huggingface_hub import notebook_login, whoami
notebook_login()

In [ ]:
print(whoami())

{'type': 'user', 'id': '61728a033edf4cc38a81237a', 'name': 'joanitolopo', 'fullname': 'Joanito Agili Lopo', 'email': 'amalopo99@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': 1746057599, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/1652231681579-61728a033edf4cc38a81237a.jpeg', 'orgs': [{'type': 'org', 'id': '643e51141d0e956d94baaade', 'name': 'kreasof-ai', 'fullname': 'Kreasof AI', 'email': None, 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/63b6f2e752c02ae8acbaa4d8/r_B-tRS854Qo5EOURHxL4.png', 'roleInOrg': 'write', 'isEnterprise': False}, {'type': 'org', 'id': '65129257d4556f9e6504afed', 'name': 'SEACrowd', 'fullname': 'SEACrowd', 'email': 'seacrowd.research@gmail.com', 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/616cd2e040e2f69baa1c7af2/n6b0McWLIRdksum9KE90i.png', 'roleInOrg': 'contributor', 'isEnterpris

In [ ]:
from datasets import load_dataset, Dataset, Audio, DatasetDict, config
import pandas as pd
from tqdm import tqdm
from librosa import load, get_duration
from tqdm.notebook import tqdm
from transformers import (
    WhisperFeatureExtractor, WhisperForConditionalGeneration, WhisperTokenizer,
    WhisperProcessor, Seq2SeqTrainingArguments, Seq2SeqTrainer, BitsAndBytesConfig, AutoModelForSpeechSeq2Seq
)
import torch
from torch.utils.data import IterableDataset
from datasets import IterableDatasetDict, Audio
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
import os
from peft import (
    TaskType,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
)

import transformers
from transformers import TrainerCallback
from transformers.trainer_pt_utils import IterableDatasetShard
from torch.utils.data import IterableDataset
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
# from accelerate import Accelerator

In [ ]:
# Set the whole variables use
# ======== Model Argument ============
model_name_or_path = "distil-whisper/distil-large-v3"
load_in_8bit = False
use_lora = False
lora_r = 32
lora_alpha = 64
lora_dropout = 0.05
lora_target_modules= ["q_proj", "v_proj"]
bias = "none"
language = "en"
task = "transcribe"

# ======== Data Argument ============
data_path = "kreasof-ai/en-hi-IWSLT2025"

# ======== Training Argument ============
push_to_hub = True
hf_repo = "kreasof-ai"
output_dir = f"{model_name_or_path.split('/')[1]}-en2hi"
do_normalize_eval = True
do_lower_case = False
do_remove_punctuation = False
# use_accelerator = True

cache_dir = "/content/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True)

# Set ALL Hugging Face related cache directories
os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_dir, "transformers")
os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_dir, "datasets")
os.environ["HF_HOME"] = os.path.join(cache_dir, "hf_home")
os.environ["HF_ASSETS_CACHE"] = os.path.join(cache_dir, "assets")
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.join(cache_dir, "hub")
os.environ["HF_MODULES_CACHE"] = os.path.join(cache_dir, "modules")

# Create all directories
for dir_path in [os.environ["TRANSFORMERS_CACHE"],
                os.environ["HF_DATASETS_CACHE"],
                os.environ["HF_HOME"],
                os.environ["HF_ASSETS_CACHE"],
                os.environ["HUGGINGFACE_HUB_CACHE"],
                os.environ["HF_MODULES_CACHE"]]:
    os.makedirs(dir_path, exist_ok=True)

# Force datasets to use the new cache
config.HF_DATASETS_CACHE = os.environ["HF_DATASETS_CACHE"]

metric_wer = evaluate.load("wer")
metric_bleu = evaluate.load("sacrebleu")
metric_chrf = evaluate.load("chrf")

torch.set_float32_matmul_precision("high")
device = "cuda" if torch.cuda.is_available() else "cpu"
# torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
torch_dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float32

def load_model_and_tokenizer():
  quant_config = BitsAndBytesConfig(load_in_8bit=load_in_8bit)
  model = AutoModelForSpeechSeq2Seq.from_pretrained(model_name_or_path, config=quant_config if load_in_8bit else None,
                                                    low_cpu_mem_usage=True, attn_implementation="flash_attention_2",
                                                    torch_dtype=torch_dtype
                                                    ).to(device)
  feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)
  tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)
  processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

  # configure the model
  model.config.forced_decoder_ids = None
  model.config.suppress_tokens = []
  model.config.use_cache = False
  model.generation_config.language = language
  model.generation_config.task = task
  model.generation_config.cache_implementation = "static"
  model.generation_config.max_new_tokens = 256

  return model, feature_extractor, tokenizer, processor

def prepare_dataset(feature_extractor, tokenizer):
  raw_datasets = IterableDatasetDict()
  raw_datasets["train"] = load_dataset(data_path, split="train", streaming=True)
  raw_datasets["test"] = load_dataset(data_path, split="dev", streaming=True)

  raw_datasets["train"] = raw_datasets["train"].filter(lambda x: x["text_en"] is not None and x["text_hi"] is not None and x["audio"] is not None)
  raw_datasets["test"] = raw_datasets["test"].filter(lambda x: x["text_en"] is not None and x["text_hi"] is not None and x["audio"] is not None)
  raw_datasets = raw_datasets.cast_column("audio", Audio(sampling_rate=16000))

  def tokenize_function(batch):
    audio = batch["audio"]

    #compute log-Me1 input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["text_en"],
                                max_length=448,  # Ensure target length is within model limits
                                truncation=True,
                                padding="max_length").input_ids
    return batch

  dataset = raw_datasets.map(tokenize_function, remove_columns=list(next(iter(raw_datasets.values())).features)).with_format("torch")
  return dataset["train"], dataset["test"]

def setup_model_for_training(model):
  if load_in_8bit:
    model = prepare_model_for_kbit_training(model)
  else:
    model.enable_input_require_grads()

  if use_lora:
    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=lora_target_modules,
        lora_dropout=lora_dropout,
        bias=bias,
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

  return model

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        batch["input_features"] = batch["input_features"].to(torch.bfloat16)  # Add this line

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

# trainer callback to reinitialise and reshuffle the streamable datasets at the beginning of each epoch
class ShuffleCallback(TrainerCallback):
  def on_epoch_begin(self, args, state, control, train_dataloader, **kwargs):
    if isinstance(train_dataloader.dataset, IterableDatasetShard):
      pass  # set_epoch() is handled by the Trainer
    elif isinstance(train_dataloader.dataset, IterableDataset):
      train_dataloader.dataset.set_epoch(train_dataloader.dataset._epoch + 1)

def train():
  model, feature_extractor, tokenizer, processor = load_model_and_tokenizer()
  model = setup_model_for_training(model)
  train_dataset, test_dataset = prepare_dataset(feature_extractor, tokenizer)
  data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
  normalizer = BasicTextNormalizer()

  def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    if do_normalize_eval:
        pred_str = [normalizer(pred) for pred in pred_str]
        label_str = [normalizer(label) for label in label_str]
        # filtering step to only evaluate the samples that correspond to non-zero references:
        pred_str = [pred_str[i] for i in range(len(pred_str)) if len(label_str[i]) > 0]
        label_str = [label_str[i] for i in range(len(label_str)) if len(label_str[i]) > 0]

    wer = 100 * metric_wer.compute(predictions=pred_str, references=label_str)
    bleu = metric_bleu.compute(predictions=pred_str, references=label_str)
    bleu = round(bleu["score"], 2)
    chrf = metric_chrf.compute(predictions=pred_str, references=label_str)
    chrf = round(chrf["score"], 2)

    return {"bleu": bleu, "chrf": chrf, "wer": wer}

  training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    bf16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    # generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    hub_model_id=f"{hf_repo}/{output_dir}",
    num_train_epochs=3)

  trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    callbacks=[ShuffleCallback])

  if use_lora:
    old_state_dict = model.state_dict
    model.state_dict = (
        lambda self, *_, **__: get_peft_model_state_dict(
            self, old_state_dict()
        )
    ).__get__(model, type(model))

  transformers.logging.set_verbosity_info()
  trainer.train()
  trainer.push_to_hub()

train()